In [ ]:
## CNN-RNN model

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


image_size = (125, 128)

# name classes
classes = ["your classes"]
num_classes = len(classes)
frames_per_sequence = 30


def load_data(base_path):
    X, y = [], []
    for class_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, class_name)
        for data_folder in os.listdir(class_path):
            data_path = os.path.join(class_path, data_folder)
            if os.path.isdir(data_path):
                frames = []
                for frame_file in sorted(os.listdir(data_path)):
                    if frame_file.endswith('.jpg'):
                        frame_path = os.path.join(data_path, frame_file)
                        img = load_img(frame_path, target_size=image_size, color_mode='grayscale')
                        img_array = img_to_array(img)
                        frames.append(img_array)
                if len(frames) == frames_per_sequence:
                    X.append(np.array(frames))
                    y.append(class_index)
    X = np.array(X)
    y = np.array(y)
    return X, y

base_path = 'path your dataset' 
X, y = load_data(base_path)

X = X / 255.0

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = (frames_per_sequence, *image_size, 1)
input_layer = Input(shape=input_shape)

cnn_model = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(input_layer)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Flatten())(cnn_model)

rnn_model = LSTM(128, return_sequences=False)(cnn_model)
rnn_model = Dropout(0.5)(rnn_model)
output_layer = Dense(num_classes, activation='softmax')(rnn_model)

model = Model(inputs=input_layer, outputs=output_layer)


model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.summary()


model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)


model.save('cnn_rnn_model.h5')



In [ ]:
## 3D-CNN model

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

image_size = (125, 128)
frames_per_sequence = 30
# name classes
classes = ["your classes"]
num_classes = len(classes)

def load_data(base_path):
    X, y = [], []
    for class_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, class_name)
        for data_folder in os.listdir(class_path):
            data_path = os.path.join(class_path, data_folder)
            if os.path.isdir(data_path):
                frames = []
                for frame_file in sorted(os.listdir(data_path)):
                    if frame_file.endswith('.jpg'):
                        frame_path = os.path.join(data_path, frame_file)
                        img = load_img(frame_path, target_size=image_size, color_mode='grayscale')
                        img_array = img_to_array(img)
                        frames.append(img_array)
                if len(frames) == frames_per_sequence:
                    X.append(np.array(frames))
                    y.append(class_index)
    X = np.array(X)
    y = np.array(y)
    return X, y

base_path = 'path your dataset' 
X, y = load_data(base_path)

X = X.astype('float32') / 255.0

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

input_shape = (frames_per_sequence, *image_size, 1)
input_layer = Input(shape=input_shape)

model = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(input_layer)
model = MaxPooling3D((1, 2, 2))(model)
model = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(model)
model = MaxPooling3D((2, 2, 2))(model)
model = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(model)
model = MaxPooling3D((2, 2, 2))(model)
model = Flatten()(model)
model = Dense(128, activation='relu')(model)
model = Dropout(0.5)(model)
output_layer = Dense(num_classes, activation='softmax')(model)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=8)

model.save('3d_cnn_model.h5')


In [ ]:
## CNN-RNN Model with LLM for Multifunctional Issues

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed, Dropout, Concatenate
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from transformers import TFBertModel, BertTokenizer


image_size = (125, 128)
frames_per_sequence = 30
# name classes
classes = ["your classes"]
num_classes = len(classes)

def load_data(base_path):
    X, y, texts = [], [], []
    for class_index, class_name in enumerate(classes):
        class_path = os.path.join(base_path, class_name)
        for data_folder in os.listdir(class_path):
            data_path = os.path.join(class_path, data_folder)
            if os.path.isdir(data_path):
                frames = []
                text_path = os.path.join(data_path, "text.txt")  
                with open(text_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()
                for frame_file in sorted(os.listdir(data_path)):
                    if frame_file.endswith('.jpg'):
                        frame_path = os.path.join(data_path, frame_file)
                        img = load_img(frame_path, target_size=image_size, color_mode='grayscale')
                        img_array = img_to_array(img)
                        frames.append(img_array)
                if len(frames) == frames_per_sequence:
                    X.append(np.array(frames))
                    y.append(class_index)
                    texts.append(text)
    X = np.array(X)
    y = np.array(y)
    return X, y, texts

base_path = 'path your dataset' 
X, y, texts = load_data(base_path)

X = X.astype('float32') / 255.0

X_train, X_val, y_train, y_val, texts_train, texts_val = train_test_split(X, y, texts, test_size=0.2, random_state=42)


input_shape = (frames_per_sequence, *image_size, 1)
input_layer = Input(shape=input_shape)

cnn_model = TimeDistributed(Conv2D(32, (3, 3), activation='relu'))(input_layer)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(64, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Conv2D(128, (3, 3), activation='relu'))(cnn_model)
cnn_model = TimeDistributed(MaxPooling2D((2, 2)))(cnn_model)
cnn_model = TimeDistributed(Flatten())(cnn_model)

rnn_model = LSTM(128, return_sequences=False)(cnn_model)
rnn_model = Dropout(0.5)(rnn_model)
 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

text_inputs = tokenizer(texts_train, return_tensors='tf', padding=True, truncation=True, max_length=128)
input_ids = Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = Input(shape=(128,), dtype=tf.int32, name='attention_mask')

bert_outputs = bert_model(input_ids, attention_mask=attention_mask)
text_features = bert_outputs.last_hidden_state[:, 0, :]  


combined = Concatenate()([rnn_model, text_features])
combined = Dense(128, activation='relu')(combined)
combined = Dropout(0.5)(combined)
output_layer = Dense(num_classes, activation='softmax')(combined)

model = Model(inputs=[input_layer, input_ids, attention_mask], outputs=output_layer)

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()